In [14]:
import os
print(os.path.exists('external/pydominion/dominion'))
print(os.path.exists('external/pydominion/dominion/Game.py'))

True
True


In [15]:
import sys
print(sys.path)

['c:\\Users\\rharr\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'c:\\Users\\rharr\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'c:\\Users\\rharr\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'c:\\Users\\rharr\\AppData\\Local\\Programs\\Python\\Python313', '', 'C:\\Users\\rharr\\AppData\\Roaming\\Python\\Python313\\site-packages', 'C:\\Users\\rharr\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32', 'C:\\Users\\rharr\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32\\lib', 'C:\\Users\\rharr\\AppData\\Roaming\\Python\\Python313\\site-packages\\Pythonwin', 'c:\\Users\\rharr\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages', 'c:\\Users\\rharr\\OneDrive\\Documents\\GitHub\\Dominating_Dominion', 'c:\\Users\\rharr\\OneDrive\\Documents\\GitHub\\Dominating_Dominion\\path\\to\\external\\pydominion']


In [4]:
# Basic needs
import sys
import os

# Add the path to the parent directory of the 'dominion' package
# Update this path to where your dominion package is actually located
sys.path.append(os.path.abspath('.\\external\\pydominion'))

# Then you can import like this
from dominion.Game import Game

In [3]:
import gymnasium as gym
#from external.pydominion.dominion.Game import Game
import sys
# For maskable PPO
from sb3_contrib.ppo_mask import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
# For DQN
from stable_baselines3 import DQN

# For recursion error, colorama was causing issues, dont worry about this
#import sys
#import os
#os.environ['CLICOLOR'] = '0'

# Outstanding Considerations

**Environment Questions**
- How long is a good limit for number of turns? An average game lasts 15-40 turns
- For our action space, the number of valid actions seems to be a max of 23... should we change?

**Results we want from training an agent**
- A graph of Reward vs Iteration for each player
- Some sort of graphs to show agent training, such as loss
- See what the deck looks like at the end of a game

**For training timestep values:**
- Quick test: 25,000
- Short trainings (learn basic moves maybe): 100,000
- Intermediate (start learning some strategy): 500,000
- Full Training (needed for decent gameplay): 1,000,000

# Play the game with agent
User Input
- Change `model_dir` as needed to location of where model was saved. Should be log_dir + model_name from the `run_masked_ppo.py` script
- If needed, change the cell with the comment "Load the model" to work for masked_ppo vs dqn
- If needed, change the cell with the comment "Play the game!" to work for masked_ppo vs dqn

In [5]:
model_dir = "logs/Rylan/masked_ppo_2_v2-2/ppo_masked_dominion"

In [6]:
# Register gym environment
gym.register(
    id="Dominion-v1",
    entry_point="DominionEnv:DominionEnv"
)

In [7]:
# Make the environment
env = gym.make("Dominion-v1", 
         num_players=2, 
         card_set=["Cellar", "Market", "Militia", "Mine", "Moat", 
                   "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
         quiet_flag = False,
         debug_flag = False
        )

In [8]:
# Load the model (switch as needed)
model = MaskablePPO.load(model_dir)
#model = DQN.load(model_dir)

In [9]:
# Play the game!
obs, _ = env.reset()
while True:
    #action, _ = model.predict(obs) # Use for DQN
    action, _ = model.predict(obs, action_masks=env.unwrapped.get_action_mask()) # Use for masked_ppo
    #print(f"Predicted action: {action}") # Use for debugging
    obs, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break

ALL: Playing with Copper
ALL: Playing with Silver
ALL: Playing with Gold
ALL: Playing with Estate
ALL: Playing with Duchy
ALL: Playing with Province
ALL: Playing with Cellar
ALL: Playing with Market
ALL: Playing with Militia
ALL: Playing with Mine
ALL: Playing with Moat
ALL: Playing with Remodel
ALL: Playing with Smithy
ALL: Playing with Throne Room
ALL: Playing with Village
ALL: Playing with Workshop
ALL: Playing with Artifacts
Scott: Dealt Copper
Scott: Dealt Estate
Scott: Dealt Estate
Scott: Dealt Copper
Scott: Dealt Estate
ALL: Player Scott is at the table
Judith: Dealt Estate
Judith: Dealt Copper
Judith: Dealt Copper
Judith: Dealt Copper
Judith: Dealt Copper
ALL: Player Judith is at the table
Judith: 
Judith: --------------------------------------------------
Judith: | Phase: Start
Judith: | Tokens: Journey Faceup
Judith: | Hand (5): Copper, Copper, Copper, Copper, Estate
Judith: | Played: <NONE>
Judith: | Deck Size: 5
Judith: | Discard (0): 
Judith: | Trash (0): 
Judith: | 0 card

# Train with Maskable PPO
See documentation [here](https://sb3-contrib.readthedocs.io/en/master/modules/ppo_mask.html)

- One timestep = one call to "step" function
- One episode = one game played
- Since PPO runs in chuncks (2048 per iteration) then 2048 (chunck size) / 583 (time it take to run one episode) = 3.5 iterations played per chunck

User input:
- NONE

This is just a testing grounds if you need it

In [39]:
#Register
gym.register(
    id="Dominion-v1",
    entry_point="DominionEnv:DominionEnv"
)

# Make environment space
env = gym.make("Dominion-v1", 
         num_players=2, 
         card_set=["Cellar", "Market", "Militia", "Mine", "Moat", 
                   "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
         quiet_flag=True,
         debug_flag = True
        )

# Mask the environment so that it only includes valid action choices
env = ActionMasker(env, lambda env: env.get_action_mask())

In [ ]:
# Recursion error from running in notebook, dont worry about this
#import os
#import sys
#os.environ['CLICOLOR'] = '0'
#sys.setrecursionlimit(1000000000)
#print(sys.getrecursionlimit())

In [ ]:
# Each thing takes 2,000 timesteps
# Episode length is around 565
log_path = "logs/PPO_masked_training_output1.log"
with open(log_path, "w") as f:
    sys.stdout = f
    sys.stderr = f

    # Train the model
    model = MaskablePPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=25000, reset_num_timesteps=False)
    print("Done learning")

# Reset stdout/stderr back to normal (very important)
sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

print(f"Training finished. Logs are in: {log_path}")
model.save("test_ppo_masked_cartpole")

# Train with Regular DQN
See documentation [here](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html)

**User input**:
- NONE

This is just a testing grounds if you need it

In [ ]:
#Register
gym.register(
    id="Dominion-v1",
    entry_point="DominionEnv:DominionEnv"
)

# Make environment space
env = gym.make("Dominion-v1", 
         num_players=2, 
         card_set=["Cellar", "Market", "Militia", "Mine", "Moat", 
                   "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
         quiet_flag=True,
         debug_flag = True
        )

In [ ]:
sys.stdout = open("logs/DQN_training_output1.log", "w")
sys.stderr = sys.stdout

# Train the model
model = DQN("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=250000, reset_num_timesteps=False)
print("Done learning")
sys.stdout.close()
print("Closed output file")

# Save the model
model.save("dqn_cartpole")
print("Saved model")

# Test anything within pydominion as needed
**User input:**
- NONE

This is just a testing grounds if you need it

In [ ]:
game = Game(
    numplayers=2,
    initcards=["Cellar", "Market", "Militia", "Mine", "Moat", 
                "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
    validate_only=False,
    prosperity=False,
    potions = False,
    shelters = False,
    card_path="external/pydominion/dominion/cards"
)

In [ ]:
# Start game to have something
game.start_game()
# Get the options, usually helpful to look into
options = game.current_player._choice_selection()